In [1]:
from pyoculus.problems import AnalyticCylindricalBfield
from pyoculus.solvers import PoincarePlot, FixedPoint, Manifold
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
separatrix = {"type": "circular-current-loop", "amplitude": -10, "R": 6, "Z": -5.5}
maxwellboltzmann = {"m": 6, "n": -1, "d": np.sqrt(2), "type": "maxwell-boltzmann", "amplitude": 0.5}

# Creating the pyoculus problem object, adding the perturbation here use the R, Z provided as center point
pyoproblem = AnalyticCylindricalBfield.without_axis(
    6,
    0,
    0.91,
    0.6,
    perturbations_args=[separatrix],
    Rbegin=1,
    Rend=8,
    niter=800,
    guess=[6.41, -0.7],
    tol=1e-9,
)

# # Adding perturbation after the object is created uses the found axis as center point
pyoproblem.add_perturbation(maxwellboltzmann)

### Finding the X-point
print("\nFinding the X-point\n")

# set up the integrator for the FixedPoint
iparams = dict()
iparams["rtol"] = 1e-13

pparams = dict()
pparams["nrestart"] = 0
pparams["niter"] = 300

# set up the FixedPoint object
fixedpoint = FixedPoint(pyoproblem, pparams, integrator_params=iparams)

# find the X-point
guess = [6.21560891, -4.46981856]
print(f"Initial guess: {guess}")

fixedpoint.compute(guess=guess, pp=0, qq=1, sbegin=4, send=9, tol=1e-10)

if fixedpoint.successful:
    results = [list(p) for p in zip(fixedpoint.x, fixedpoint.y, fixedpoint.z)]
else:
    raise ValueError("X-point not found")

# Set up the manifold
iparams = dict()
iparams["rtol"] = 1e-13
manifold = Manifold(fixedpoint, pyoproblem, integrator_params=iparams)

# Choose the tangles to work with
manifold.choose(0, 0)

0 - dr : 0.010422282084043258
1 - RZ : [ 6.41409394 -0.69368043]
1 - dr : 6.293042617255254e-06
2 - RZ : [ 6.41409781 -0.69367863]
2 - dr : 9.776471937205409e-12
0 - dr : 9.776471937205409e-12

Finding the X-point

Initial guess: [6.21560891, -4.46981856]
0 - [DeltaR, DeltaZ] : [0.38046432 0.30396059] - dtheta : 0.10487729978217719
0 - [StepR, StepZ]: [-0.0229111  -0.02508285]
1 - RZ : [ 6.19269781 -4.49490141] - rhotheta : [ 3.80766497 -1.62897501]
1 - [DeltaR, DeltaZ] : [-0.18204545 -0.19238585] - dtheta : 0.04250251705208874
1 - [StepR, StepZ]: [0.00314814 0.00763012]
2 - RZ : [ 6.19584596 -4.48727128] - rhotheta : [ 3.79986567 -1.62826468]
2 - [DeltaR, DeltaZ] : [-0.02187519 -0.03756228] - dtheta : 0.005127217976363951
2 - [StepR, StepZ]: [ 0.00772472 -0.01071993]
3 - RZ : [ 6.20357068 -4.49799122] - rhotheta : [ 3.81013332 -1.62607901]
3 - [DeltaR, DeltaZ] : [0.00064998 0.00029984] - dtheta : 0.0001659980452346499
3 - [StepR, StepZ]: [ 9.32458354e-05 -2.16309921e-04]
4 - RZ : [ 6.

In [ ]:
fig = pickle.load(open("manifold_05131749.pkl", "rb"))
ax = fig.gca()

In [ ]:
eps_s_1, eps_u_1 = manifold.find_homoclinic()
bounds_1 = manifold.find_bounds(eps_s_1, eps_u_1)

In [ ]:
bounds_i = np.array(bounds_1)
bounds_i[0][0] = manifold.clinics[-1][1]
bounds_i[1][1] = manifold.clinics[-1][2]
bounds_i = (tuple(bounds_i[0]), tuple(bounds_i[1]))

In [ ]:
bounds_1

In [ ]:
bounds_i

In [ ]:
# i = 1
fund = manifold.fundamental_segment
# guess_i = [bounds_1[0][1]*np.power(manifold.lambda_s, 5/6), bounds_1[1][0]*np.power(manifold.lambda_u, 5/6)]
guess_i = [bounds_1[0][1]*np.power(manifold.lambda_s, 1/2), bounds_1[1][0]*np.power(manifold.lambda_u, 1/2)]
print(f"initial guess: {guess_i}")

In [ ]:
eps_s_i, eps_u_i = manifold.find_homoclinic(*guess_i, bounds = bounds_i, n_s=7, n_u=5, options={"factor": 0.1})

In [3]:
manifold.find_clinics(n_points=2, tol=1e-6)
manifold.order()

2024-05-22 17:48:41 [info     ] Search for minimum of the linear error succeeded, epsilon = 2.05960e-07
2024-05-22 17:48:41 [info     ] Search for minimum of the linear error succeeded, epsilon = 3.00363e-07
2024-05-22 17:48:41 [debug    ] Guess - 2.841422693592067e-06, 4.1438140663202e-06
2024-05-22 17:48:41 [debug    ] Bounds - ((2.059596983864675e-07, 5.888805085570602e-06), (3.003634794400747e-07, 8.587991612080473e-06))
2024-05-22 17:48:41 [debug    ] n_s, n_u - 7, 6
2024-05-22 17:48:41 [debug    ] Inside : (2.8414226935920685e-06, 4.1438140663202e-06)
2024-05-22 17:48:41 [debug    ] Returns - (array([ 8.6620776 , -0.24890767]), array([5.22138836, 1.74885734]), array([ 3.44068923, -1.99776501]))
2024-05-22 17:48:41 [debug    ] Inside : (2.8414226935920685e-06, 4.1438140663202e-06)
2024-05-22 17:48:41 [debug    ] Returns - (array([ 8.6620776 , -0.24890767]), array([5.22138836, 1.74885734]), array([ 3.44068923, -1.99776501]))
2024-05-22 17:48:41 [debug    ] Inside : (2.8414226935920

In [4]:
manifold.resonance_area()

2024-05-22 17:48:53 [info     ] Forward integration goes beyond stable saddle point.
2024-05-22 17:48:53 [debug    ] rfp: [ 6.20366396 -4.49820757], rz_end: [ 6.20361761 -4.49824962], rz: [ 6.20366165 -4.49820782]
2024-05-22 17:48:54 [info     ] Backward integration goes beyond stable saddle point.
2024-05-22 17:48:54 [debug    ] rfp: [ 6.20366396 -4.49820757], rz_end: [ 6.20365991 -4.49820038], rz: [ 6.20366456 -4.49820665]
2024-05-22 17:48:54 [info     ] Potential integration completed for homo/hetero-clinic point of order : 9.979e-07
2024-05-22 17:48:54 [info     ] Forward integration goes beyond stable saddle point.
2024-05-22 17:48:54 [debug    ] rfp: [ 6.20366396 -4.49820757], rz_end: [ 6.2035725  -4.49829066], rz: [ 6.20366066 -4.4982103 ]
2024-05-22 17:48:54 [info     ] Backward integration goes beyond stable saddle point.
2024-05-22 17:48:54 [debug    ] rfp: [ 6.20366396 -4.49820757], rz_end: [ 6.20366772 -4.49821418], rz: [ 6.20366416 -4.49820774]
2024-05-22 17:48:54 [info   

(array([ 1.85531923, -1.85531923]),
 [[array([-14.36038823,   4.51808968, -19.8946102 ,   2.25619221,
           11.87700978,  12.40673787,  12.42571079]),
   array([  1.15664594,  28.97561846,  -0.8736151 , -11.84123443,
          -12.40552216, -12.42560757])],
  [array([ -9.53345766,  -2.97013097, -12.7710893 ,  -8.56856494,
           10.29454014,  12.34552231,  12.42350996,  12.42635225]),
   array([ 22.72298733,   8.17294405, -10.95023577, -12.37241051,
          -12.42445204, -12.42627951])]],
 [[[array([3.9907224 , 0.66340198]),
    array([ 4.46536617, -2.25323211]),
    array([ 5.87095146, -4.00637149]),
    array([ 6.18764011, -4.47039472]),
    array([ 6.20309556, -4.49720851]),
    array([ 6.20364402, -4.49817265]),
    array([ 6.20366165, -4.49820782])],
   [array([ 8.43741136, -2.30709314]),
    array([ 6.57630221, -4.18311605]),
    array([ 6.2207122 , -4.48279917]),
    array([ 6.20426753, -4.49765936]),
    array([ 6.20368508, -4.49818838]),
    array([ 6.20366456, -4.4

In [5]:
manifold.areas

array([[ 1.85531923e+00,  1.31340775e-03,  3.81710285e-05],
       [-1.85531923e+00,  1.31340775e-03,  3.81710285e-05]])

In [3]:
manifold.find_homoclinic(2.179774577375764e-07, 3.84907705931755e-06, n_s=7, n_u=6)
manifold.find_homoclinic(1.2560522331587575e-06, 2.388481018696467e-05, n_s=7, n_u=5)

manifold.order()

2024-05-22 13:09:43 [debug    ] Guess - 2.179774577375764e-07, 3.84907705931755e-06
2024-05-22 13:09:43 [debug    ] Bounds - ((2.179774577375764e-07, 6.233109701299461e-06), (3.84907705931755e-06, 0.00011005805429358986))
2024-05-22 13:09:43 [debug    ] n_s, n_u - 7, 6
2024-05-22 13:09:43 [debug    ] Inside : (2.179774577375764e-07, 3.84907705931755e-06)
2024-05-22 13:09:45 [debug    ] Returns - (array([5.41601047, 1.96063451]), array([5.41601048, 1.96063451]), array([-1.12521814e-09, -1.59716684e-10]))
2024-05-22 13:09:45 [debug    ] Inside : (2.179774598991796e-07, 3.849077055417897e-06)
2024-05-22 13:09:46 [debug    ] Returns - (array([5.41601049, 1.96063451]), array([5.41601048, 1.96063451]), array([1.43470427e-08, 7.52905649e-09]))
2024-05-22 13:09:46 [info     ] Root finding status : The solution converged.
2024-05-22 13:09:46 [debug    ] Root finding object :  message: The solution converged.
 success: True
  status: 1
     fun: [-1.125e-09 -1.597e-10]
       x: [-1.534e+01 -1.2

In [ ]:
manifold.clinics

In [ ]:
marker = ["X", "o", "s", "p", "P", "*", "x", "D", "d", "^", "v", "<", ">"]
for i, clinic in enumerate(manifold.clinics):
    eps_s_i, eps_u_i = clinic[1:3]

    n_s, n_u = 10, 10
    
    hs_i = manifold.integrate(manifold.rfp_s + eps_s_i * manifold.vector_s, n_s, -1)
    hu_i = manifold.integrate(manifold.rfp_u + eps_u_i * manifold.vector_u, n_u, 1)
    # ax.scatter(hs_i[0,:], hs_i[1,:], marker=marker[i], color="purple", zorder=10)
    ax.scatter(hu_i[0,:], hu_i[1,:], marker=marker[i], color="royalblue", edgecolor='cyan', zorder=10, label=f'$h_{i+1}$')

In [ ]:
fig.set_size_inches(12, 12)
ax.set_xlim(3.5, 9.2)
ax.set_ylim(-6, 2.5)
ax.set_title('')
ax.set_xlabel(r'R [m]')
ax.set_ylabel(r'Z [m]')

ax.legend()


fig

In [ ]:
pickle.dump(fig, open("perturbed-6-1-0.1-general.pkl", "wb"))

In [ ]:
eps_s, eps_u = manifold.fundamental_segment
eps_s, eps_u = eps_s[0], eps_u[0]

In [ ]:
manifold.compute(eps_s=eps_s, eps_u=eps_u, neps=40)

In [ ]:
rh = manifold.integrate_single(manifold.rfp_u + manifold.clinics[1][0]*manifold.vector_u, 3, 1, ret_jacobian=False)

In [ ]:
default = {
    "markersize": 2,
    "fmt": "-x",
    "colors": ["red", "blue"],
}

area_calc = []
dirdict = [manifold.unstable["+"], manifold.stable["+"]]

for i, out in enumerate(dirdict):
    tmpcolor = default["colors"][i]
    
    if i == 0:
        area_calc.append(out[44:80, 3])
        out = out[44:80, 3]
    else:
        area_calc.append(out[:42, 9])
        out = out[:42, 9]

    out = out.T.flatten()

    if i==0:
        out = np.concatenate((rh, out))
    else:
        out = np.concatenate((out, rh))

    ax.plot(
        out[::2],
        out[1::2],
        default["fmt"],
        color='black',
        markersize=6,
        linewidth=2
    )

In [ ]:
path1 = area_calc[0].reshape(-1,2)
path2 = area_calc[1].reshape(-1,2)

path1 = np.vstack((rh, path1))
path2 = np.vstack((path2, rh))

In [ ]:
path1.T

In [ ]:
path2.T

In [ ]:
# Concatenate the paths to form a loop
loop = np.vstack((path1[1:], path2[1:]))

x = loop[:, 0]
y = loop[:, 1]
area = 0.5*np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

In [ ]:
ax.scatter(6.7913, -4.0464, color='black', label='$B^\phi$ evaluation point')

In [ ]:
ax.legend()
ax.set_xlim(6, 7.3)
ax.set_ylim(-4.6, -3.6)
fig

In [ ]:
pickle.dump(fig, open("perturbed-6-1-0.1-closeup.pkl", "wb"))

In [ ]:
pyoproblem.B([6.7913, 0., -4.0464])

In [ ]:
flux1 = area * pyoproblem.B([6.7913, 0., -4.0464])[1] * 6.7913
flux1

#### Area by computation

In [4]:
areas, potential_integrations, history = manifold.resonance_area()

2024-05-22 13:09:51 [info     ] Forward integration goes beyond stable saddle point.
2024-05-22 13:09:51 [debug    ] rfp: [ 6.20363263 -4.49636071], rz_end: [ 6.2036283  -4.49636463], rz: [ 6.20363237 -4.49636066]
2024-05-22 13:09:51 [info     ] Backward integration goes beyond stable saddle point.
2024-05-22 13:09:51 [debug    ] rfp: [ 6.20363263 -4.49636071], rz_end: [ 6.20364463 -4.49638169], rz: [ 6.20363315 -4.49636135]
2024-05-22 13:09:51 [info     ] Potential integration completed for homo/hetero-clinic point of order : 3.849e-06
2024-05-22 13:09:51 [info     ] Forward integration goes beyond stable saddle point.
2024-05-22 13:09:51 [debug    ] rfp: [ 6.20363263 -4.49636071], rz_end: [ 6.20363013 -4.49636292], rz: [ 6.20363192 -4.4963597 ]
2024-05-22 13:09:51 [info     ] Backward integration goes beyond stable saddle point.
2024-05-22 13:09:51 [debug    ] rfp: [ 6.20363263 -4.49636071], rz_end: [ 6.20363578 -4.49636621], rz: [ 6.20363276 -4.49636088]
2024-05-22 13:09:51 [info   

In [5]:
areas

array([-0.38894751,  0.38894751])

In [6]:
manifold.areas

array([[-3.88947513e-01,  5.76281936e-04,  2.99349844e-05],
       [ 3.88947513e-01,  5.76281936e-04,  2.99349844e-05]])

In [ ]:
(flux1 - manifold.areas[0])/flux1

### Errors and convergence

In [ ]:
areas

In [ ]:
potential_integrations

In [ ]:
fig, ax = plt.subplots()
for pot in potential_integrations:
    ax.scatter(1+np.arange(7), pot[0]-pot[1])